## Importing all the necessary dependencies

In [1]:
import pandas as pd 
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

## Reading the CSV files

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
3017,0,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""nam...",NaN,10984,"[{""id"": 3698, ""name"": ""mobbing""}, {""id"": 3836,...",en,Valentine,Five friends are stalked and murdered by a mas...,5.904126,"[{""name"": ""Village Roadshow Pictures"", ""id"": 7...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2001-02-01,0,96.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Remember that kid everyone ignored on Valentin...,Valentine,5.0,114
451,80000000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""nam...",NaN,11618,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,The Haunting,"Dr. David Marrow invites Nell Vance, and Theo ...",19.375768,"[{""name"": ""DreamWorks"", ""id"": 7}, {""name"": ""Ro...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-23,91188905,113.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Some houses are born bad.,The Haunting,5.2,369
3556,15000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 878, ""...",http://www.10cloverfieldlane.com/,333371,"[{""id"": 1930, ""name"": ""kidnapping""}, {""id"": 23...",en,10 Cloverfield Lane,"After a car accident, Michelle awakens to find...",53.698683,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-03-10,108286421,103.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Monsters come in many forms.,10 Cloverfield Lane,6.8,2468
2686,0,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""nam...",NaN,10008,"[{""id"": 616, ""name"": ""witch""}, {""id"": 10183, ""...",en,An American Haunting,Based on the true events of the only case in U...,7.436315,"[{""name"": ""Media Pro Pictures"", ""id"": 3244}, {...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2005-05-05,0,83.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Possession Knows No Bounds,An American Haunting,5.3,142
4446,850000,"[{""id"": 35, ""name"": ""Comedy""}]",NaN,84200,"[{""id"": 9964, ""name"": ""crude humor""}, {""id"": 1...",en,"For a Good Time, Call...","College ""frenemies"" Lauren and Katie move in t...",14.238376,"[{""name"": ""AdScott Pictures"", ""id"": 52241}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-08-31,1243961,85.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Lose your hang-ups. Find your calling.,"For a Good Time, Call...",5.8,110


In [4]:
credits.sample(5)

,movie_id,title,cast,crew
1978,20697,Ready to Rumble,"[{""cast_id"": 1, ""character"": ""Gordie Boggs"", ""...","[{""credit_id"": ""52fe43f4c3a368484e007d43"", ""de..."
3215,71805,Shipwrecked,"[{""cast_id"": 1000, ""character"": ""Haakon Haakon...","[{""credit_id"": ""57798e709251415fbe0018d5"", ""de..."
1578,10096,13 Going on 30,"[{""cast_id"": 12, ""character"": ""Jenna Rink"", ""c...","[{""credit_id"": ""52fe43259251416c750054a3"", ""de..."
1803,31640,Blood and Wine,"[{""cast_id"": 1, ""character"": ""Alex Gates"", ""cr...","[{""credit_id"": ""532813d19251411a22001e36"", ""de..."
3172,6521,The Contender,"[{""cast_id"": 1, ""character"": ""Sheldon Runyon"",...","[{""credit_id"": ""59991a7fc3a3686d67007a52"", ""de..."


In [5]:
movies = movies.merge(credits, on='title') 

In [6]:
movies.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
4194,1530000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 18, ""nam...",NaN,104,"[{""id"": 220, ""name"": ""berlin""}, {""id"": 585, ""n...",de,Lola rennt,Lola receives a phone call from her boyfriend ...,27.365620,"[{""name"": ""X-Filme Creative Pool"", ""id"": 1972}...",...,81.0,"[{""iso_639_1"": ""de"", ""name"": ""Deutsch""}, {""iso...",Released,Every second of every day you're faced with a ...,Run Lola Run,7.2,664,104,"[{""cast_id"": 11, ""character"": ""Lola"", ""credit_...","[{""credit_id"": ""569edf60925141370b000700"", ""de..."
4729,100000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""nam...",http://thegallowsmovie.com/,299245,"[{""id"": 2125, ""name"": ""gallows""}, {""id"": 6270,...",en,The Gallows,20 years after a horrific accident during a sm...,18.045782,"[{""name"": ""New Line Cinema"", ""id"": 12}, {""name...",...,87.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Every School Has Its Spirit,The Gallows,4.9,361,299245,"[{""cast_id"": 0, ""character"": ""Cassidy"", ""credi...","[{""credit_id"": ""54951e769251413af7001c60"", ""de..."
1201,40000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.mgm.com/view/movie/1132/The-Living-...,708,"[{""id"": 212, ""name"": ""london england""}, {""id"":...",en,The Living Daylights,James Bond helps a Russian General escape into...,23.331459,"[{""name"": ""United Artists"", ""id"": 60}, {""name""...",...,130.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Licensed to thrill.,The Living Daylights,6.2,440,708,"[{""cast_id"": 18, ""character"": ""James Bond"", ""c...","[{""credit_id"": ""52fe426ec3a36847f801df5f"", ""de..."
1003,49000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 878, ""na...",NaN,10796,"[{""id"": 851, ""name"": ""dual identity""}]",en,The One,A sheriff's deputy fights an alternate univers...,21.746245,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,87.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Stealing the power of the universes one by one.,The One,5.7,437,10796,"[{""cast_id"": 13, ""character"": ""Gabe Law / Gabr...","[{""credit_id"": ""598ee621c3a368747300f38f"", ""de..."
2533,15000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",NaN,17202,[],en,Why Did I Get Married?,The film is about the difficulty of maintainin...,2.005111,"[{""name"": ""Lions Gate Films"", ""id"": 35}]",...,113.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Because we complete each other's sentences.,Why Did I Get Married?,6.1,33,17202,"[{""cast_id"": 2, ""character"": ""Terry"", ""credit_...","[{""credit_id"": ""52fe470f9251416c7508c81d"", ""de..."


In [7]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [8]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
4454,45767,Time Changer,The year is 1890 and Bible professor Russell C...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 4379, ""name"": ""time travel""}, {""id"": 5...","[{""cast_id"": 3, ""character"": ""Russell Carlisle...","[{""credit_id"": ""52fe46d2c3a36847f8114123"", ""de..."
4486,14275,Hoop Dreams,This documentary follows two inner-city Chicag...,"[{""id"": 99, ""name"": ""Documentary""}]","[{""id"": 520, ""name"": ""chicago""}, {""id"": 1483, ...","[{""cast_id"": 1, ""character"": ""Himself"", ""credi...","[{""credit_id"": ""52fe45e19251416c75065927"", ""de..."
4279,52790,The Basket,"Peter Coyote (E.T., Erin Brokovich) and Karen ...","[{""id"": 18, ""name"": ""Drama""}]",[],"[{""cast_id"": 1, ""character"": ""Martin Conlon"", ...","[{""credit_id"": ""52fe4842c3a36847f815e997"", ""de..."
740,8078,Alien: Resurrection,"Two hundred years after Lt. Ripley died, a gro...","[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...","[{""id"": 803, ""name"": ""android""}, {""id"": 3070, ...","[{""cast_id"": 1, ""character"": ""Ripley"", ""credit...","[{""credit_id"": ""53957a4dc3a3686434003f9d"", ""de..."
2853,34549,Max Keeble's Big Move,"Max Keeble, the victim of his 7th grade class,...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...",[],"[{""cast_id"": 1, ""character"": ""Max Keeble"", ""cr...","[{""credit_id"": ""52fe456a9251416c91031963"", ""de..."


## Data Preprocessing

In [9]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [10]:
movies.dropna(inplace=True)

In [11]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [12]:
movies.duplicated().sum()

0

In [13]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [14]:
def create_genre_list_of_movie(movie_genre_object_array):
    
    genre_name_list = []
    
    for i in ast.literal_eval(movie_genre_object_array):
        
        
        genre_name_list.append(i['name'])
        
        
    return genre_name_list

In [15]:
movies['genres'] = movies['genres'].apply(create_genre_list_of_movie)

In [16]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
778,11260,Meet Dave,A crew of miniature aliens operate a spaceship...,"[Comedy, Science Fiction, Adventure, Family]","[{""id"": 242, ""name"": ""new york""}, {""id"": 1316,...","[{""cast_id"": 14, ""character"": ""Dave Ming Chang...","[{""credit_id"": ""52fe441c9251416c75029207"", ""de..."
2896,4170,Trade,"A Texas cop (Kevin Kline), whose own daughter ...","[Drama, Thriller]","[{""id"": 279, ""name"": ""usa""}, {""id"": 572, ""name...","[{""cast_id"": 11, ""character"": ""Ray Sheridan"", ...","[{""credit_id"": ""52fe43b0c3a36847f8067d21"", ""de..."
3221,272693,The DUFF,Bianca's universe turns upside down when she l...,"[Romance, Comedy]","[{""id"": 6270, ""name"": ""high school""}, {""id"": 1...","[{""cast_id"": 2, ""character"": ""Bianca Piper"", ""...","[{""credit_id"": ""549c98e1c3a3680b27003697"", ""de..."
1266,9544,FearDotCom,"With four corpses on his hands, New York City ...","[Horror, Thriller]","[{""id"": 4776, ""name"": ""race against time""}, {""...","[{""cast_id"": 1, ""character"": ""Detective Mike R...","[{""credit_id"": ""52fe4507c3a36847f80b81d5"", ""de..."
757,87825,Trouble with the Curve,"Slowed by age and failing eyesight, crack base...","[Drama, Romance]","[{""id"": 515, ""name"": ""women""}, {""id"": 1480, ""n...","[{""cast_id"": 15, ""character"": ""Gus"", ""credit_i...","[{""credit_id"": ""52fe49da9251416c910baf1d"", ""de..."


In [17]:
def create_keyword_list_of_movie(movie_keyword_object_array):
    
    keyword_name_list = []
    
    for i in ast.literal_eval(movie_keyword_object_array):
        
        keyword_name_list.append(i['name'])
        
        
    return keyword_name_list

In [18]:
movies['keywords'] = movies['keywords'].apply(create_keyword_list_of_movie)

In [19]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
1372,8065,21,"Ben Campbell is a young, highly intelligent, s...","[Drama, Crime]","[gambling, card game, professor, harvard unive...","[{""cast_id"": 1, ""character"": ""Ben Campbell"", ""...","[{""credit_id"": ""52fe4490c3a36847f809d381"", ""de..."
4284,46415,Neal 'n' Nikki,"The film, as its name implies, centres on Gurn...","[Comedy, Romance]",[musical],"[{""cast_id"": 4, ""character"": ""Gurneal \""Neal\""...","[{""credit_id"": ""52fe46f6c3a36847f811c031"", ""de..."
1832,168530,Ride Along,"For the past two years, high-school security g...","[Action, Comedy]","[police operation, police officer, brother-in-...","[{""cast_id"": 3, ""character"": ""James"", ""credit_...","[{""credit_id"": ""571b4198c3a36824a3000b35"", ""de..."
3047,10050,Get Over It,"When Berke Landers, a popular high school bask...","[Comedy, Romance]","[theatre play, theatre group, high school, fal...","[{""cast_id"": 19, ""character"": ""Kelly Woods"", ""...","[{""credit_id"": ""52fe43139251416c7500298f"", ""de..."
699,10708,Daddy Day Care,Two men get laid off and have to become stay-a...,"[Comedy, Family]","[competition, success, kindergarten, children,...","[{""cast_id"": 1, ""character"": ""Charlie Hinton"",...","[{""credit_id"": ""52fe43a69251416c75018bc9"", ""de..."


In [20]:
def create_top_three_cast_list_of_movie(cast_object_array):
    
    top_three_cast_name_list = []
    
    counter = 0
    
    for i in ast.literal_eval(cast_object_array):
        
        if counter != 3:
            
            top_three_cast_name_list.append(i['name'])
            
            counter += 1
        
        else:
            
            break

        
    return top_three_cast_name_list

In [21]:
movies['cast'] = movies['cast'].apply(create_top_three_cast_list_of_movie)

In [22]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
234,49519,The Croods,The Croods is a prehistoric comedy adventure t...,"[Adventure, Animation, Comedy, Family, Fantasy]","[stone age, daughter, father, prehistoric, anc...","[Nicolas Cage, Emma Stone, Ryan Reynolds]","[{""credit_id"": ""52fe4799c3a36847f813e52f"", ""de..."
486,274854,The Last Witch Hunter,"The modern world holds many secrets, but by fa...","[Fantasy, Action, Adventure]","[new york, witch, uprising, witch hunter]","[Vin Diesel, Rose Leslie, Elijah Wood]","[{""credit_id"": ""56a4febdc3a368388b0079f2"", ""de..."
3544,979,Irreversible,Events over the course of one traumatic night ...,"[Drama, Thriller, Crime, Mystery]","[paris, prostitute, rape, sex, nudity, trauma,...","[Monica Bellucci, Vincent Cassel, Albert Dupon...","[{""credit_id"": ""56e9ec4ec3a3682d630047cc"", ""de..."
1220,41446,Scream 4,"Sidney Prescott, now the author of a self-help...","[Horror, Mystery]","[sheriff, book, knife, horror, rescue, author,...","[David Arquette, Neve Campbell, Courteney Cox]","[{""credit_id"": ""52fe45cac3a36847f80da9e1"", ""de..."
252,40805,The Green Hornet,"Britt Reid (Seth Rogen), the heir to the large...","[Action, Crime, Comedy]","[bomb, martial arts, assassin, vandalism, nigh...","[Seth Rogen, Jay Chou, Christoph Waltz]","[{""credit_id"": ""52fe45a4c3a36847f80d25a9"", ""de..."


In [23]:
def crew_list_of_movie(crew_object_array):
    
    crew_name_list = []
    
    for i in ast.literal_eval(crew_object_array):
        
        if i['job'] == 'Director':
            
            crew_name_list.append(i['name'])
            
            break
            
            # since every row represents a movie and every movie has only 1 director, therefore, as soon as we get the 
            # name of the director, we break out of the loop
            
    return crew_name_list

In [24]:
movies['crew'] = movies['crew'].apply(crew_list_of_movie)

In [25]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
2335,10013,Peggy Sue Got Married,Peggy Sue faints at a Highschool reunion. When...,"[Comedy, Drama, Fantasy, Romance]","[time travel, high school reunion]","[Kathleen Turner, Nicolas Cage, Barry Miller]",[Francis Ford Coppola]
196,38055,Megamind,Bumbling supervillain Megamind finally defeats...,"[Animation, Action, Comedy, Family, Science Fi...","[saving the world, date, prison, secret identi...","[Will Ferrell, Brad Pitt, Tina Fey]",[Tom McGrath]
1127,87826,Here Comes the Boom,A high school biology teacher moonlights as a ...,[Comedy],"[high school teacher, prize money, physics tea...","[Salma Hayek, Kevin James, Henry Winkler]",[Frank Coraci]
1985,40688,Meet the Deedles,Two surfers end up as Yellowstone park rangers...,"[Animation, Comedy, Family]",[],"[Paul Walker, John Ashton, Dennis Hopper]",[Steve Boyum]
4140,22600,Gangster's Paradise: Jerusalema,This South African movie tracks the rise of a ...,"[Drama, Action, Crime, Foreign]","[hearing, gang of thieves, organized crime]","[Daniel Buckland, Robert Hobbs, Rapulana Seiph...",[Ralph Ziman]


In [26]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [27]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
4200,43947,I Spit on Your Grave,"[A, remake, of, the, 1979, controversial, cult...","[Thriller, Crime, Horror]","[rape, fondling, revenge, writer, strangulatio...","[Sarah Butler, Daniel Franzese, Chad Lindberg]",[Steven R. Monroe]
2501,34647,Enter the Void,"[This, psychedelic, tour, of, life, after, dea...","[Fantasy, Drama]","[prostitute, hallucination, strip club, afterl...","[Nathaniel Brown, Paz de la Huerta, Cyril Roy]",[Gaspar Noé]
2933,12837,The Secret Life of Bees,"[Set, in, South, Carolina, in, 1964,, this, is...",[Adventure],[woman director],"[Dakota Fanning, Queen Latifah, Jennifer Hudson]",[Gina Prince-Bythewood]
1872,9655,She's the Man,"[Viola, Johnson, is, in, a, real, jam., Compli...","[Comedy, Drama, Family, Romance]","[roommate, twin sister, sport, boarding school...","[Amanda Bynes, Channing Tatum, Laura Ramsey]",[Andy Fickman]
1920,6978,Big Trouble in Little China,"[When, trucker,, Jack, Burton, agreed, to, tak...","[Action, Adventure, Comedy, Fantasy]","[kung fu, chinatown, magic, mystery]","[Kurt Russell, Kim Cattrall, Dennis Dun]",[John Carpenter]


In [28]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(' ', '') for i in x])

In [29]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
3620,12484,The Forsaken,"[A, young, man, is, in, a, race, against, time...","[Action, Adventure, Horror, Thriller]","[vampire, fullmoon, poster, hitchhiker, homoer...","[KerrSmith, BrendanFehr, IzabellaMiko]",[J.S.Cardone]
4151,18808,Highway,"[Jack, is, caught, with, the, wife, of, his, e...","[Action, Adventure, Drama]",[roadmovie],"[JakeGyllenhaal, SelmaBlair, JaredLeto]",[JamesCox]
3653,51384,"Lovely, Still","[A, holiday, fable, that, tells, the, story, o...","[Drama, Romance]",[independentfilm],"[MartinLandau, EllenBurstyn, AdamScott]",[NicholasFackler]
381,49852,The Nutcracker: The Untold Story,"[Set, in, 1920's, Vienna,, this, a, tale, of, ...","[Fantasy, Action, Family]",[],"[ElleFanning, NathanLane, JohnTurturro]",[AndreiKonchalovsky]
1850,10923,Agent Cody Banks,"[Recruited, by, the, U.S., government, to, be,...",[Action],"[spy, cia, killerrobot, delinquent, teenspy]","[FrankieMuniz, HilaryDuff, AngieHarmon]",[HaraldZwart]


In [30]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [31]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew,tags
2090,9682,Pulse,"[When, their, computer, hacker, friend, accide...","[Horror, Thriller]","[suicide, hacker, deathofafriend, website, rem...","[KristenBell, IanSomerhalder, ChristinaMilian]",[JimSonzero],"[When, their, computer, hacker, friend, accide..."
1736,17834,A Simple Wish,"[Murray, is, a, male, fairy, godmother,, and, ...","[Adventure, Comedy, Family, Fantasy]","[magic, wish, kids, fairygodmother]","[MartinShort, KathleenTurner, MaraWilson]",[MichaelRitchie],"[Murray, is, a, male, fairy, godmother,, and, ..."
4008,14139,Timecrimes,"[A, man, accidentally, gets, into, a, time, ma...","[ScienceFiction, Thriller]","[scissors, radio, nudity, time, woods, surreal...","[KarraElejalde, CandelaFernández, BárbaraGoenaga]",[NachoVigalondo],"[A, man, accidentally, gets, into, a, time, ma..."
105,241259,Alice Through the Looking Glass,"[In, the, sequel, to, Tim, Burton's, ""Alice, i...",[Fantasy],"[basedonnovel, clock, queen, sequel, aliceinwo...","[JohnnyDepp, MiaWasikowska, AnneHathaway]",[JamesBobin],"[In, the, sequel, to, Tim, Burton's, ""Alice, i..."
4452,84200,"For a Good Time, Call...","[College, ""frenemies"", Lauren, and, Katie, mov...",[Comedy],"[crudehumor, bestfriend, phonesex, innuendo, d...","[AriGraynor, LaurenMiller, MarkWebber]",[JamieTravis],"[College, ""frenemies"", Lauren, and, Katie, mov..."


In [32]:
movies = movies[['movie_id', 'title', 'tags']]

In [33]:
movies

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [34]:
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

In [35]:
movies.sample(5)

,movie_id,title,tags
945,4421,G.I. Jane,A female Senator succeeds in enrolling a woman...
718,8914,Deep Blue Sea,On a remote former submarine refueling facilit...
3524,57022,Sunshine State,A woman and her new husband returns to her hom...
2513,771,Home Alone,Eight-year-old Kevin McCallister makes the mos...
320,855,Black Hawk Down,When U.S. Rangers and an elite Delta Force tea...


In [36]:
movies['tags'] = movies['tags'].apply(lambda x: x.lower())

In [37]:
movies.sample(5)

,movie_id,title,tags
2137,14359,Doubt,"in 1964, a catholic school nun questions a pri..."
1531,606,Out of Africa,out of africa tells the story of the life of d...
286,9268,Eraser,u.s. marshall john kruger erases the identitie...
1377,11459,Sky High,set in a world where superheroes are commonly ...
2336,194662,Birdman,a fading actor best known for his portrayal of...


## Applying Vectorization and Cosine Similarity

In [38]:
ps = PorterStemmer()

In [39]:
def stem_words(text):
    
    y = []
    
    for i in text.split():
        
        y.append(ps.stem(i))
        
    return " ".join(y) 

In [40]:
movies['tags'] = movies['tags'].apply(stem_words)

In [41]:
movies.sample(5)

,movie_id,title,tags
3038,10090,Reno 911!: Miami,a rag-tag team of reno cop are call in to save...
2730,70829,The Last Godfather,"young-goo the son of mafia boss don carini, is..."
993,9923,Domino,"the daughter of actor, laurenc harvey turn awa..."
2987,16781,Madea's Family Reunion,base upon tyler perry' acclaim stage productio...
331,68737,Seventh Son,"john gregory, who is a seventh son of a sevent..."


In [42]:
vectorizer = CountVectorizer(max_features=5000, stop_words='english')

In [43]:
vector_of_tags = vectorizer.fit_transform(movies['tags']).toarray()

In [44]:
similarity_matrix = cosine_similarity(vector_of_tags)

## Recommending top 5 movies for a given movie name

In [45]:
def recommend_top_5_similar_movies_for_the_given_movie_input(movie_name):
    
    index_of_the_movie = movies[movies['title'] == movie_name].index[0]
    
    all_distances = similarity_matrix[index_of_the_movie]
    
    top_five_movies = sorted(list(enumerate(all_distances)), reverse=True, key=lambda x:x[1])[1:6] 

    for i in top_five_movies:
        
        print(movies.iloc[i[0]].title)
    

In [46]:
recommend_top_5_similar_movies_for_the_given_movie_input('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


## Creating a pickle file out of the cleaned dataset and the similarity matrix

In [47]:
pickle.dump(movies.to_dict(), open('cleaned_movies_dataset_dictionary.pkl', 'wb'))

In [48]:
pickle.dump(similarity_matrix, open('similarity_matrix.pkl', 'wb'))